In [1]:
# Import the required libraries & modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [2]:
import os
os.chdir("C:\\Users\\sriva\\Downloads\\TMC")

In [3]:
gbc_patients = pd.read_excel("working_gbc.xlsx")

In [4]:
gbc_patients.head()

,Thyroidism,Diabetes,HTN,Family_cancer_history,ECOG_final,Gall_stone_final,Pathology,Tumour_grade_final,Cancer_presentation_final,Tumour_stage_final,...,BMI_DETAILS,BILIRUBIN CONJUGATED,BILIRUBIN UNCONJUGATED,BILIRUBIN TOTAL,AST - SGOT,ALT - SGPT,ALK PHOS [ALP],GAMMA GT,ALBUMIN,Diabetes_final
0,No,No,Yes,No,1,Yes,Adenocarcinoma,Grade 3,Incidental,Stage IV,...,24.93,0.0,0.5,0.5,23.0,28.0,147.0,21.0,4.7,Yes
1,No,No,No,No,1,Not known,Adenocarcinoma,Grade X,Metastatic,Stage IV,...,22.55,0.0,1.1,0.4,348.0,78.0,206.0,481.0,3.2,No
2,No,No,Yes,No,2,Yes,Adenocarcinoma,Grade X,Metastatic,Stage IV,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,No,Yes,Yes,Not known,1,Yes,MiNEN,Grade 2,Incidental,Stage II,...,20.22,0.0,1.0,1.0,36.0,31.0,108.0,21.0,4.6,Yes
4,Hypothyroidism,Not known,Yes,Not known,Not known,Yes,Adenocarcinoma,Grade X,Locally advanced,Not known,...,24.15,12.1,2.0,20.5,286.0,130.0,492.0,209.0,2.6,Not known


In [5]:
gbc_patients.shape

(486, 26)

In [6]:
gbc_patients["Status"].value_counts()

Dead     425
Alive     34
Live      26
No         1
Name: Status, dtype: int64

In [7]:
#removing the observation having "Status" = No
gbc_patients = gbc_patients[gbc_patients['Status'] != 'No']

In [8]:
gbc_patients.shape

(485, 26)

In [9]:
##MISSING VALUES
gbc_patients.isnull().sum()

Thyroidism                    20
Diabetes                      18
HTN                           18
Family_cancer_history         20
ECOG_final                    17
Gall_stone_final              20
Pathology                      0
Tumour_grade_final            25
Cancer_presentation_final     32
Tumour_stage_final            46
Mets_site_final               36
Surgery_final                 18
Treatment_final               34
Status                         0
AGE                            0
GENDER                         0
BMI_DETAILS                   88
BILIRUBIN CONJUGATED         182
BILIRUBIN UNCONJUGATED       182
BILIRUBIN TOTAL              182
AST - SGOT                   182
ALT - SGPT                   183
ALK PHOS [ALP]               182
GAMMA GT                     183
ALBUMIN                      183
Diabetes_final                18
dtype: int64

In [10]:
## CATEGORICAL FEATURES - MERGING MULTIPLE CATEGORIES TO A FEW IN ORDER TO REDUCE THE NUMBER OF CATEGORIES

In [11]:
# Filter columns with object data type
categorical_columns = gbc_patients.select_dtypes(include=['object']).columns

# Calculate counts of each category in categorical features
category_counts = {}

for column in categorical_columns:
    counts = gbc_patients[column].value_counts()
    category_counts[column] = counts

# Print the counts of each category in categorical features
for column, counts in category_counts.items():
    print(f'Counts for {column}:')
    print(counts)
    print()

Counts for Thyroidism:
No                347
Not known          73
Hypothyroidism     45
Name: Thyroidism, dtype: int64

Counts for Diabetes:
No           274
Yes          121
Not known     72
Name: Diabetes, dtype: int64

Counts for HTN:
No           228
Yes          173
Not known     66
Name: HTN, dtype: int64

Counts for Family_cancer_history:
Not known    241
No           177
Yes           47
Name: Family_cancer_history, dtype: int64

Counts for ECOG_final:
1            260
2             98
Not known     38
3             38
0             18
4             16
Name: ECOG_final, dtype: int64

Counts for Gall_stone_final:
Yes          200
Not known    132
No           129
Polyp          4
Name: Gall_stone_final, dtype: int64

Counts for Pathology:
Adenocarcinoma                  414
Carcinoma                        34
Adenosquamous carcinoma          14
Neuroendocrine carcinoma          8
Others                            7
Mucinous adenocarcinoma           3
ICPN with invasive carcinom

In [12]:
## PATHOLOGY

In [13]:
# List of categories to keep
categories_to_keep = ['Adenocarcinoma', 'Adenosquamous carcinoma', 'Neuroendocrine carcinoma']

# Replace categories other than those to keep with "Others"
gbc_patients['Pathology'] = gbc_patients['Pathology'].where(gbc_patients['Pathology'].isin(categories_to_keep), 'Others')

In [14]:
gbc_patients['Pathology'].value_counts()

Adenocarcinoma              414
Others                       49
Adenosquamous carcinoma      14
Neuroendocrine carcinoma      8
Name: Pathology, dtype: int64

In [15]:
### Metastasis

In [20]:
'''
Liver	Extension into liver ± other organs such as lung or bone
Peritoneum & Omentum	Omentum, Peritoneal or Peritoneum, 
Lungs	Lung 
Others	Adrenal Gland, Bone, Brain, Ovary, Uterus, Stomach
	Portsite,  Hypochondrium
'''

'\nLiver\tExtension into liver ± other organs such as lung or bone\nPeritoneum & Omentum\tOmentum, Peritoneal or Peritoneum, \nLungs\tLung \nOthers\tAdrenal Gland, Bone, Brain, Ovary, Uterus, Stomach\n\tPortsite,  Hypochondrium\n'

In [19]:
# List of categories to keep
gbc_patients['Mets_site_final'] = gbc_patients['Mets_site_final'].replace(['Liver', 'Liver & Lungs', 'Liver & Bone'], 'Liver or Liver+others(lungs,bones)')


# List of categories to keep
categories_to_keep = ['Not applicable','Liver or Liver+others(lungs,bones)', 'Peritoneum & Omentum', 'Lungs']

# Replace categories other than those to keep with "Others"
gbc_patients['Mets_site_final'] = gbc_patients['Mets_site_final'].where(gbc_patients['Mets_site_final'].isin(categories_to_keep), 'Others')


gbc_patients['Mets_site_final'].value_counts()


Not applicable                        202
Liver or Liver+others(lungs,bones)    162
Others                                 69
Peritoneum & Omentum                   34
Lungs                                  18
Name: Mets_site_final, dtype: int64

In [ ]:
gbc_patients.to

In [21]:
import pandas as pd
from tabulate import tabulate

# Assuming you have already defined gbc_patients DataFrame and imported tabulate

# Filter columns with object data type
categorical_columns = gbc_patients.select_dtypes(include=['object']).columns

# Calculate counts of each category in categorical features
category_counts = {}

for column in categorical_columns:
    counts = gbc_patients[column].value_counts()
    category_counts[column] = counts

# Beautify and print the counts of each category in categorical features
for column, counts in category_counts.items():
    print(f'Counts for {column}:')
    print(tabulate(counts.reset_index(), headers=[column, 'Count'], tablefmt='grid'))
    print()


Counts for Thyroidism:
+----+----------------+---------+
|    | Thyroidism     |   Count |
+====+================+=========+
|  0 | No             |     347 |
+----+----------------+---------+
|  1 | Not known      |      73 |
+----+----------------+---------+
|  2 | Hypothyroidism |      45 |
+----+----------------+---------+

Counts for Diabetes:
+----+------------+---------+
|    | Diabetes   |   Count |
+====+============+=========+
|  0 | No         |     274 |
+----+------------+---------+
|  1 | Yes        |     121 |
+----+------------+---------+
|  2 | Not known  |      72 |
+----+------------+---------+

Counts for HTN:
+----+-----------+---------+
|    | HTN       |   Count |
+====+===========+=========+
|  0 | No        |     228 |
+----+-----------+---------+
|  1 | Yes       |     173 |
+----+-----------+---------+
|  2 | Not known |      66 |
+----+-----------+---------+

Counts for Family_cancer_history:
+----+-------------------------+---------+
|    | Family_cancer_hi

In [23]:
gbc_patients.to_excel('sem2_working_gbc_reduced_categories.xlsx', index=False)

In [ ]:
## DATASET WITH LESS NUMBER OF CATEGORIES IN SOME CATEGORICAL FEATURES.